# Pitch Extraction Simulation notebook 

This notebook is for testing of simulation blocks

In [74]:
import pyaudio
import struct
import matplotlib.pyplot as plt
import matplotlib.animation as animation
# from matplotlib import style
import numpy as np
import sys, select, os
import time
import csv
%matplotlib qt

FORMAT = pyaudio.paInt16
CHANNELS = 1 # Mono audio
SAMPLE_RATE = 44100 # Sample rate
WINDOW_SAMPLES = 2048 # Each window will be 2048 samples long at 22.68us per sample = 0.046s per window
WINDOWS_PER_BUFFER = 1
FRAMES_PER_BUFFER = WINDOW_SAMPLES*WINDOWS_PER_BUFFER

In [63]:
p = pyaudio.PyAudio()
streamIn = p.open(
        format = FORMAT,
        channels = CHANNELS,
        rate = SAMPLE_RATE,
        input = True,
        output = False,
        frames_per_buffer = FRAMES_PER_BUFFER
        )

In [64]:
# Slower method
# %timeit data_int = struct.unpack(str(FRAMES_PER_BUFFER)+'h',data) 

### AMDF Pitch extractor function

In [65]:
# Function is defined as Dn = 1/L * sum^{L}_{k=0} |S_{j} - S{j-n}| with n = 0,1,2,3... 
#                                                              and Sj are the samples
def amdf_PE(inputWindow):
    D_tau = np.zeros((8,256))
    minIndices = np.zeros(8)
    freq = np.zeros(8)
    vol = np.zeros(8)
#     scale_factor = 1/len(inputWindow)
    
    for c in range(8):
        inputWindow_block = inputWindow[c*256:(c+1)*256]
        tau = np.arange(0,(len(inputWindow_block)-1))
        for i,t in enumerate(tau):
            temp = 0
            shifted = np.zeros_like(inputWindow_block)
            if t == 0:
                shifted = inputWindow_block
            else:
                shifted[:t] = 0
                shifted[t:] = inputWindow_block[:-t]
    #         for i,win in enumerate(inputWindow):
    #         temp =+ np.abs(win-shifted[i])
            diffArr = np.abs(np.subtract(inputWindow_block,shifted))
            temp = (np.sum(diffArr)/len(inputWindow_block))
            D_tau[c,i] = temp
        offset = np.argmax(D_tau[c,:])
        minIndices[c] = (c*256+offset)+np.argmin(D_tau[c,offset:-1])
        freq[c] = SAMPLE_RATE/(minIndices[c]-(c*256))
        vol[c] = (np.mean(np.abs(inputWindow_block))/2**15)
        
#     minIndices[0] = np.argmin(D_tau[1:1024])
#     minIndices[1] = 1024+np.argmin(D_tau[1025:2048])
#     minIndices[2] = 2048+np.argmin(D_tau[2049:4096])
#     print(minIndices)
#     for i, data in enumerate(D_tau): 
#         try:
#             if (D_tau[i-1]>data) and (D_tau[i+1]> data):
#                 minIndices.append(i)
#         except:
#             if (D_tau[i-1])>data: 
#                 minIndices.append(i)
    return D_tau.flatten(),minIndices, freq, vol

In [38]:
from sklearn.preprocessing import normalize
x = np.random.rand(100)

norm2 = normalize(x[:,np.newaxis], axis = 0).ravel()
plt.plot(x)
plt.plot(norm1)
plt.plot(norm2)

In [23]:
a = np.array([19,2,13,45,5,6,7,28,9,5,5,5,31,11,1,52,5,5])
vals, count = np.unique(a,return_counts = True)
index = count.argmax()
vals[index]

5

### Stretch function
Trying to determine an on-the-fly function generator that will compensate for the decay of the AMDF. The linear function does not work as the decay is not linear and approximation with a linear function does not yield good results yet

Possible fixes:
    - scaling factor in the denominator
    - non-linear function (will require some form of interpolation)

In [6]:
## Stretch function (not currently working)
D_max = np.max(D_tau)
stretch = np.zeros(FRAMES_PER_BUFFER)
stretch[0] = 1
for n in range(1,FRAMES_PER_BUFFER):
    stretch[n] = D_max*((-1/(10*2*FRAMES_PER_BUFFER))*n)
D_tau = (D_tau*stretch-(D_max/2))

NameError: name 'D_tau' is not defined

In [43]:
plt.plot(stretch)

In [7]:
data = streamIn.read(FRAMES_PER_BUFFER)
data_int = np.frombuffer(data,dtype='<i2')
# print(data_int)
a = time.time()
AMDF_data = amdf_PE(data_int)
b = time.time()
print("Time taken: {}".format(b-a))
# plt.plot(AMDF_data)

Time taken: 0.026920795440673828


### Plotting function

In [79]:
fig, ax = plt.subplots()
ax.set_title("AMDF")
ax.set_ylim([-(2**8),(2**12)])
vline0 = ax.axvline(x =   0.,color = 'red')
vline01 = ax.axvline(x =   0.,color = 'red')
vline1 = ax.axvline(x = 256,color = 'k')
vline01 = ax.axvline(x = 256,color = 'k')
vline2 = ax.axvline(x = 512,color = 'green')
vline02 = ax.axvline(x = 512,color = 'green')
vline3 = ax.axvline(x = 768,color = 'orange')
vline03 = ax.axvline(x = 768,color = 'orange')
vline4 = ax.axvline(x = 1024,color = 'pink')
vline04 = ax.axvline(x = 1024,color = 'pink')
vline5 = ax.axvline(x = 1280,color = 'purple')
vline05 = ax.axvline(x = 1280,color = 'purple')
vline6 = ax.axvline(x = 1536,color = 'yellow')
vline06 = ax.axvline(x = 1536,color = 'yellow')
vline7 = ax.axvline(x = 1792,color = 'cyan')
vline07 = ax.axvline(x = 1792,color = 'cyan')
line, = ax.plot(np.zeros(FRAMES_PER_BUFFER))

# line1, = ax.plot(np.zeros(FRAMES_PER_BUFFER))
# line2, = ax.plot(np.zeros(FRAMES_PER_BUFFER))
plt.show()
signal = []
amdf_val = []
stretch = []
freqs = []
vols = []
timerStart = time.time()
timer = 0
while True:
    data = streamIn.read(FRAMES_PER_BUFFER)
    data_int2 = np.frombuffer(data,dtype='<i2')
    amdf_data, vline, freqArr, vol = amdf_PE(data_int2)
    line.set_ydata(amdf_data)
    freqs.append(freqArr)
    vols.append(vol)
    timer = time.time()-timerStart
    if timer>= 3.0:
        finalTimer = time.time()-timerStart
        break
    vline0.set_data(vline[0] , [0,1])
    vline1.set_data(vline[1] , [0,1])
    vline2.set_data(vline[2] , [0,1])
    vline3.set_data(vline[3] , [0,1])
    vline4.set_data(vline[4] , [0,1])
    vline5.set_data(vline[5] , [0,1])
    vline6.set_data(vline[6] , [0,1])
    vline7.set_data(vline[7] , [0,1])
    fig.canvas.draw()
    fig.canvas.update()
    fig.canvas.flush_events()
# plt.close(fig)
freqs = np.array(freqs)
freqs = freqs.flatten()
vols = np.array(vols)
vols = vols.flatten()
vols /= np.max(vols) 
print(finalTimer)
xAxis = np.round(np.arange(0,timer,timer/8 ),2)
xticker = np.arange(len(freqs))
fig, ax1 = plt.subplots()
fig.suptitle("Frequency progression of recorded audio")
ax1.set_ylabel("Frequency (Hz)", color = "green")
ax1.set_xlabel("Time (sampled at {} Hz)".format(SAMPLE_RATE))
ax1.plot(freqs, color = 'green')

ax2 = ax1.twinx()
ax2.set_ylabel("Volume as percentage of max (%)", color = "red")
ax2.plot(vols*100, color = 'red')
plt.xticks(xticker[::66], xAxis)

3.006865978240967


([<matplotlib.axis.XTick at 0x1570018a748>,
 [Text(0, 0, '0.0'),
  Text(0, 0, '0.38'),
  Text(0, 0, '0.75'),
  Text(0, 0, '1.13'),
  Text(0, 0, '1.5'),
  Text(0, 0, '1.88'),
  Text(0, 0, '2.26'),
  Text(0, 0, '2.63')])

In [81]:
with open('frequency_list.csv','w') as file1:
    np.savetxt(file1, freqs, fmt = '%10.3f', delimiter = ',')

with open('volume_list.csv','w') as file2:
    np.savetxt(file2, vols, fmt= '%10.3f', delimiter = ',')

In [82]:
xAxis

array([0.  , 0.38, 0.75, 1.13, 1.5 , 1.88, 2.26, 2.63])

## Plotting BBBW output for accuracy and speed testing

In [77]:
print(freqs)

[1575.         1575.         1575.         1575.         1575.
 1575.         1575.         1575.         1575.         1575.
 1575.         1575.         1575.         1575.         1520.68965517
  177.10843373  177.10843373 1575.          787.5         173.62204724
  184.51882845  189.27038627  173.62204724  179.26829268 2450.
  173.62204724  173.62204724  175.          173.62204724  223.85786802
  173.62204724  188.46153846  173.62204724  253.44827586  173.62204724
  254.9132948   254.9132948   254.9132948   254.9132948   253.44827586
  262.5         253.44827586  256.39534884  253.44827586  256.39534884
  254.9132948   256.39534884  257.89473684  259.41176471  256.39534884
  257.89473684  256.39534884  257.89473684  257.89473684  259.41176471
  259.41176471  223.85786802  250.56818182  223.85786802  243.64640884
  196.875       173.62204724  173.62204724  191.73913043  188.46153846
  173.62204724  373.72881356  267.27272727  469.14893617  194.27312775
  173.62204724  173.62204724  